# Vectors

In [69]:
x = Int8[5, 7, 3] # Makes a vector with entries 5, 7, 3 all of type Int8

3-element Vector{Int8}:
 5
 7
 3

In [70]:
# A vector consists of 3 pieces of inversion:
# element type, length, address memory (called pointer)
eltype(x), length(x), pointer(x)

(Int8, 3, Ptr{Int8} @0x000002019c3739f8)

In [71]:
# DONT DO THIS: may cause Julia to crash if you access outside allowed memore
p = pointer(x)
Base.unsafe_load(p+2) # address + 2 gives 3rd entry, since each element uses 8-bits = 1 byte

3

In [72]:
x = Int16[5, 7, 3] # now each element is an Int16, which uses 16-bits = 2 bytes

3-element Vector{Int16}:
 5
 7
 3

In [73]:
p = pointer(x)
Base.unsafe_load(p + 1) # nonsense: we are mixing up two bytes from two different numbers

1792

In [74]:
T = eltype(x) # type of each entry Int16
s = sizeof(T) # 2 (the number of bytes for each entry)
Base.unsafe_load(p + s) # next entry, which is two addresses away from first

7

In [75]:
x[2] = 11

11

In [76]:
# we can modify an arrays entries as follows, changing the data in memory:
x[2] = 11
x

3-element Vector{Int16}:
  5
 11
  3

## Matrices (dense)

In [77]:
# creates a 3 × 2 matrix with Int entries
A = [1 2;
     3 4;
     5 6]

3×2 Matrix{Int64}:
 1  2
 3  4
 5  6

In [78]:
# storage is same as a vector
vec(A)

6-element Vector{Int64}:
 1
 3
 5
 2
 4
 6

In [79]:
# a Matrix consists of 3 things:
# element type, size (two Ints), and a pointer to the first address 
eltype(A), size(A), pointer(A)

(Int64, (3, 2), Ptr{Int64} @0x00000201e38da810)

# Matrix-vector mult.

In [80]:
# Load LinearAlgebra which supports some linear algebra operations
using LinearAlgebra

In [81]:
# inbuilt way

n = 5
A = randn(n,n) # n x n matrix with normal entries
x = randn(n)
A*x # uses optimised routine

5-element Vector{Float64}:
  0.557478167359159
  3.0511386027668967
 -7.76818818701138
 -2.631087470119887
 -5.051212302886905

In [82]:
## Alg 1 (by rows)
function mul_rows(A, x)
    # T is usually Float64
    T = promote_type(eltype(A), eltype(x))
    m,n = size(A) # dimensions of the matrix
    if n != length(x) #!= means not-equal
        error("incompatible")
    end
    b = zeros(T, m) # vector of all zeros
    for k = 1:m
        for j = 1:n
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end

# alg 2 (by columns)
function mul_cols(A, x)
    # T is usually Float64
    T = promote_type(eltype(A), eltype(x))
    m,n = size(A) # dimensions of the matrix
    if n != length(x) #!= means not-equal
        error("incompatible")
    end
    b = zeros(T, m) # vector of all zeros
    for j = 1:n    
        for k = 1:m
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end

# algorithms match (approximately)
A*x ≈ mul_rows(A,x) ≈ mul_cols(A,x)

true

In [83]:

# mul_cols is 5–10x faster than mul_rows since it access memory in order
n = 10000
A = randn(n,n)
x = randn(n)
println("inbulit")
@time A*x
println("by rows")
@time mul_rows(A,x)
println("by cols")
@time mul_cols(A,x);

inbulit
  0.029655 seconds (2 allocations: 78.172 KiB)
by rows
  2.317815 seconds (2 allocations: 78.172 KiB)
by cols
  0.149427 seconds (2 allocations: 78.172 KiB)


# Triangular matrices

In [84]:
A = [1 2 3; 4 5 6; 7 8 9]
U = UpperTriangular(A) # wraps the matrix A

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 1  2  3
 ⋅  5  6
 ⋅  ⋅  9

In [85]:
U[1,3] = 6 # change entries of U
U

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 1  2  6
 ⋅  5  6
 ⋅  ⋅  9

In [86]:
A # modifying U also modifes A

3×3 Matrix{Int64}:
 1  2  6
 4  5  6
 7  8  9

In [87]:
fieldnames(typeof(U))
U.data

3×3 Matrix{Int64}:
 1  2  6
 4  5  6
 7  8  9

In [88]:
# UpperTriangular matrices can solve linear systems fast:
n = 10000
A = randn(n,n)
U = UpperTriangular(A)
b = randn(n)
@time U \ b # 10x faster by using back-substitution
@time A \ b; # uses matrix-factorization A = P * L *U

  0.030819 seconds (2 allocations: 78.172 KiB)
  8.709107 seconds (6 allocations: 763.092 MiB, 1.00% gc time)


In [89]:
# our implementation of U\b using backsub
function ldiv(U::UpperTriangular, b)
    T = promote_type(eltype(U), eltype(b))
    m,n = size(U) # note m == n
    x = zeros(T, n)
    for k = n:-1:1 # compute x[n] first
        r = b[k] # r is a "dummy variable"
        for j = k+1:n
            r = r - U[k,j]*x[j] # note we computed x[j] for j > k
        end
        x[k] = r/U[k,k]
    end
    x
end

ldiv (generic function with 2 methods)

In [90]:
n = 1000
# A = randn(n,n)  This is VERY INACCURATE!! WHY???
A = randn(n,n) + 100I # Adding 100I adds 100 to each diagonal of A, making it "stable"
U = UpperTriangular(A)
x = randn(n)
b = U*x

x ≈ U\b ≈ ldiv(U,b)

true

In [91]:
# The issue without the 100I is that the diagonal becomes small. We will explain this in detail later:
minimum(abs.(diag(A)))

96.94112309999423

# 3. Banded Matrices

In [92]:
d = [1,2,3,4]
D = Diagonal(d) # stored as a vector, l = u = 0

4×4 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅  ⋅
 ⋅  2  ⋅  ⋅
 ⋅  ⋅  3  ⋅
 ⋅  ⋅  ⋅  4

In [93]:
d = [1,2,3,4,5]
u = [5,6,7,8]
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer

5×5 Bidiagonal{Int64, Vector{Int64}}:
 1  5  ⋅  ⋅  ⋅
 ⋅  2  6  ⋅  ⋅
 ⋅  ⋅  3  7  ⋅
 ⋅  ⋅  ⋅  4  8
 ⋅  ⋅  ⋅  ⋅  5

In [94]:
d = [1,2,3,4,5]
u = [5,6,7,8]
l = [9,10,11,12]
T = Tridiagonal(l, d, u) # store 3 vectors

# Tridiagonal come up for solving ODEs

5×5 Tridiagonal{Int64, Vector{Int64}}:
 1   5   ⋅   ⋅  ⋅
 9   2   6   ⋅  ⋅
 ⋅  10   3   7  ⋅
 ⋅   ⋅  11   4  8
 ⋅   ⋅   ⋅  12  5

In [95]:
n = 10_000_000 # _ means ,
d = randn(n) .+ 1000 # add 1000 to diag so its "stable"
u = randn(n-1)
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer
b = randn(n)
@time U \ b; # computational cost scales like n so we can go to very large n

LoadError: OutOfMemoryError()

In [ ]:
fieldnames(typeof(U))
U.uplo # == 'U', `U.uplo` tells us if its upper or lower triangular

'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)

In [ ]:
# our implementation of U\b using backsub
function ldiv(U::Bidiagonal, b) 
    T = promote_type(eltype(U), eltype(b))
    m,n = size(U) # note m == n
    x = zeros(T, n)
    if U.uplo == 'U'
        for k = n:-1:1 # compute x[n] first
            r = b[k] # r is a "dummy variabl"
            for j = k+1:min(n,k+1) # only access non-zeros of U
                r = r - U[k,j]*x[j] # note we computed x[j] for j > k
            end
            x[k] = r/U[k,k]
        end
    else
        error("Not implemented for lower triangular matrices")
    end
    x
end

ldiv (generic function with 2 methods)

In [ ]:
n = 1000 # _ means ,
d = randn(n) .+ 1000 # add 1000 to diag so its "stable"
u = randn(n-1)
U = Bidiagonal(d, u, :U) # stores two vectors and whether upper or loweer
b = randn(n)
U \ b ≈ ldiv(U,b)

true

In [ ]:
struct Infinity <: Number # Infinity is subtype of Number
end

In [ ]:
A = randn(5, 5)

5×5 Matrix{Float64}:
  0.550474     0.64461    0.250931   1.09789     0.30505
  0.383918    -0.364992  -0.641957   0.275132    0.173818
 -0.00336948   0.474827  -1.83163    0.0918589  -0.25316
  1.48203      0.868203  -0.22489   -0.856924   -0.859044
  0.068598     2.31288   -0.432712  -0.507873   -0.0590507

In [ ]:
min.(A,0.4)

5×5 Matrix{Float64}:
  0.4          0.4        0.250931   0.4         0.30505
  0.383918    -0.364992  -0.641957   0.275132    0.173818
 -0.00336948   0.4       -1.83163    0.0918589  -0.25316
  0.4          0.4       -0.22489   -0.856924   -0.859044
  0.068598     0.4       -0.432712  -0.507873   -0.0590507

In [ ]:
struct MinMatrix{T} <: AbstractMatrix{T}
  A::Matrix{T}
  c::T
end

import Base: size, getindex
size(M::MinMatrix) = size(M.A)

# getindex(M, k, j) = M[k, j]
getindex(M::MinMatrix, k::Int, j::Int) = min(M.A[k, j], M.c)

# setindex(M, v, k, j) = (M[k, j] = v)
function setindex!(M::MinMatrix, v, k::Int, j::Int)
  

LoadError: syntax: incomplete: premature end of input